In [553]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import xgboost as xgb

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [624]:
matches = pd.read_csv("/content/ODI.csv")


In [625]:
matches.head(10)

,Unnamed: 0,Team_1,Team_2,Winner,Margin,Ground,Match Date
0,0,Australia,England,Australia,5 wickets,Melbourne,"Jan 5, 1971"
1,1,England,Australia,England,6 wickets,Manchester,"Aug 24, 1972"
2,2,England,Australia,Australia,5 wickets,Lord's,"Aug 26, 1972"
3,3,England,Australia,England,2 wickets,Birmingham,"Aug 28, 1972"
4,4,New Zealand,Pakistan,New Zealand,22 runs,Christchurch,"Feb 11, 1973"
5,5,England,New Zealand,England,7 wickets,Swansea,"Jul 18, 1973"
6,6,England,New Zealand,no result,-,Manchester,"Jul 20, 1973"
7,7,England,West Indies,England,1 wicket,Leeds,"Sep 5, 1973"
8,8,England,West Indies,West Indies,8 wickets,The Oval,"Sep 7, 1973"
9,9,New Zealand,Australia,Australia,7 wickets,Dunedin,"Mar 30, 1974"


In [557]:
matches=matches.drop(['Ground','Unnamed: 0'], axis=1)

In [558]:
x=matches.loc[matches['Winner']=='no result'].index

In [559]:
matches=matches.drop(x,axis=0)
matches=matches.reset_index()

In [560]:
year=[]
for i in matches['Match Date']:
  sp= i.split(", ")
  year.append(sp[1])
matches['Year']=year

In [561]:
y=matches.loc[matches.Year<'2010'].index
matches=matches.drop(y,axis=0)
matches=matches.reset_index()
matches=matches.drop(['level_0','index','Match Date'],axis=1)

In [562]:
teams=['Australia', 'England', 'New Zealand','Sri Lanka' ,'Pakistan',
 'India','Bangladesh' ,'South Africa' ,'Afghanistan','Netherlands']
notTeams=[]
for i in range(0,len(matches)):
    if(not (matches.loc[i,"Team_1"] in teams) or (not (matches.loc[i,"Team_2"] in teams))):
        notTeams.append(i)
matches = matches.drop(notTeams)
matches

,Team_1,Team_2,Winner,Margin,Year
0,Bangladesh,Sri Lanka,Sri Lanka,7 wickets,2010
1,India,Sri Lanka,Sri Lanka,5 wickets,2010
2,Bangladesh,India,India,6 wickets,2010
3,Bangladesh,Sri Lanka,Sri Lanka,9 wickets,2010
4,India,Sri Lanka,India,8 wickets,2010
...,...,...,...,...,...
1574,Pakistan,New Zealand,Pakistan,5 wickets,2023
1576,Pakistan,New Zealand,Pakistan,7 wickets,2023
1578,Pakistan,New Zealand,Pakistan,26 runs,2023
1579,Pakistan,New Zealand,Pakistan,102 runs,2023


In [563]:
matches=matches.reset_index()
matches=matches.drop(['index'], axis=1)

In [564]:
wr = np.zeros((10,10))
for i in range(0,10):
    for j in range(0,10):
        if(not (i == j)):
            iwin = 0
            jwin = 0
            for index, row in matches.iterrows():
                if((row['Team_1'] == teams[i] and row['Team_2'] == teams[j]) or
                   (row["Team_2"] == teams[i] and row["Team_1"] == teams[j])):
                    if(row["Winner"] == teams[i]):
                        iwin+=int(row["Year"]) - 2009
                    if(row["Winner"] == teams[j]):
                        jwin+=int(row["Year"]) - 2009
            if(iwin + jwin > 0):
                wr[i][j] = round(100 * iwin/(iwin + jwin),2)
            else:
                wr[i][j] = -10.00
        else:
            wr[i][j] = -10.00

WinRates = pd.DataFrame(wr, index = teams, columns=teams)
WinRates = WinRates.replace(-10,'-')
WinRates

,Australia,England,New Zealand,Sri Lanka,Pakistan,India,Bangladesh,South Africa,Afghanistan,Netherlands
Australia,-,51.9,72.26,55.29,77.84,51.88,100.0,27.27,100.0,-
England,48.1,-,62.2,56.65,80.59,44.23,68.42,50.85,100.0,100.0
New Zealand,27.74,37.8,-,77.64,57.66,46.58,78.08,44.86,100.0,100.0
Sri Lanka,44.71,43.35,22.36,-,30.41,20.0,63.92,28.16,60.71,-
Pakistan,22.16,19.41,42.34,69.59,-,26.32,50.91,52.63,100.0,100.0
India,48.12,55.77,53.42,80.0,73.68,-,66.39,54.93,100.0,100.0
Bangladesh,0.0,31.58,21.92,36.08,49.09,33.61,-,51.61,65.66,66.67
South Africa,72.73,49.15,55.14,71.84,47.37,45.07,48.39,-,100.0,100.0
Afghanistan,0.0,0.0,0.0,39.29,0.0,0.0,34.34,0.0,-,93.62
Netherlands,-,0.0,0.0,-,0.0,0.0,33.33,0.0,6.38,-


In [565]:
matches.head()
final=matches.copy()

In [566]:
winrates = []
for index, row in matches.iterrows():
    Team_1 = row["Team_1"]
    Team_2 = row["Team_2"]
    count1 = 0
    count2 = 0
    for i in range(0,len(row)):
        if((matches["Team_1"][i] == Team_1 and matches["Team_2"][i] == Team_2)
          or (matches["Team_1"][i] == Team_2 and matches["Team_2"][i] == Team_1)):
            if(matches["Winner"][i] == Team_1):
                count1 += int(matches["Year"][i]) - 2009
            else:
                count2 += int(matches["Year"][i]) - 2009
    if(count1 + count2 > 0):
        winrates.append(round(count1/(count1+count2),2))
    else:
        winrates.append(0.50)
matches["WinRateVsTeam"] = winrates

In [567]:
winratesT1 = []
winratesT2=[]
for index, row in matches.iterrows():
    Team_1 = row["Team_1"]
    Team_2 = row["Team_2"]
    count1_t1 = 0
    count2_t1= 0
    count1_t2 = 0
    count2_t2= 0
    for i in range(0,len(row)):
      if(matches["Team_1"][i] == Team_1  or matches["Team_2"][i] == Team_1):
            if(matches["Winner"][i] == Team_1):
                count1_t1 += int(matches["Year"][i]) - 2009
            else:
                count2_t1 += int(matches["Year"][i]) - 2009
      if(matches["Team_1"][i] == Team_2  or matches["Team_2"][i] == Team_2):
            if(matches["Winner"][i] == Team_2):
                count1_t2 += int(matches["Year"][i]) - 2009
            else:
                count2_t2 += int(matches["Year"][i]) - 2009

    if(count1_t1 + count2_t1 > 0):
        winratesT1.append(round(count1_t1/(count1_t1+count2_t1),2))
    else:
        winratesT1.append(0.50)


    if(count1_t2 + count2_t2 > 0):
        winratesT2.append(round(count1_t2/(count1_t2+count2_t2),2))
    else:
        winratesT2.append(0.50)



matches["Team_1_WR"] = winratesT1
matches["Team_2_WR"] = winratesT2

In [568]:
d=matches.loc[matches.Winner=='tied'].index
matches=matches.drop(d,axis=0)
matches=matches.reset_index(drop=True)
final=matches

In [569]:
wtype=[]
for i in matches['Margin']:
  sp= i.split(" ")
  #if wickets then 1
  #if runs then 0
  if sp[1]=='wickets':
    wtype.append(1)
  else:
    wtype.append(0)
matches['WinByWickets']=wtype

In [570]:
margin = []
for index,row in matches.iterrows():
    if(not isinstance(row["Margin"],float)):
        margin.append(row["Margin"].split()[0])
    else:
        margin.append(0)
matches["Margin"] = margin

In [571]:
matches

,Team_1,Team_2,Winner,Margin,Year,WinRateVsTeam,Team_1_WR,Team_2_WR,WinByWickets
0,Bangladesh,Sri Lanka,Sri Lanka,7,2010,0.0,0.00,0.75,1
1,India,Sri Lanka,Sri Lanka,5,2010,0.5,0.75,0.75,1
2,Bangladesh,India,India,6,2010,0.0,0.00,0.75,1
3,Bangladesh,Sri Lanka,Sri Lanka,9,2010,0.0,0.00,0.75,1
4,India,Sri Lanka,India,8,2010,0.5,0.75,0.75,1
...,...,...,...,...,...,...,...,...,...
789,Pakistan,New Zealand,Pakistan,5,2023,0.5,0.50,0.50,1
790,Pakistan,New Zealand,Pakistan,7,2023,0.5,0.50,0.50,1
791,Pakistan,New Zealand,Pakistan,26,2023,0.5,0.50,0.50,0
792,Pakistan,New Zealand,Pakistan,102,2023,0.5,0.50,0.50,0


In [572]:
match_pred=matches.copy()

In [573]:
i=0
for index,row in match_pred.iterrows():
    if(row["Winner"] == row["Team_1"]):
        match_pred["Winner"][i] = 0
    elif(row["Winner"] == row["Team_2"]):
        match_pred["Winner"][i] = 1
    i+=1

In [574]:
match_pred = match_pred.drop("Margin",axis="columns")
match_pred=match_pred.drop("WinByWickets",axis="columns")

In [575]:
#stats for each team
teamData = pd.read_csv("/content/Team_Stats.csv")
teamData=teamData.drop(['Unnamed: 0'],axis=1)
teamData

,Team,MatPlayed,High_inns_score,NO,BatAvg,Bat_SR,Runs/Inns,BF/Inns,Hundreds/Inns,Fifties/Inns,Boundries/Inns,Sixes/Inns,FieldingDiss/Inns,Stumping,BallAvg,Ball_SR,Econ,RunsConceded/Inns,WickTaken/Inns
0,India,1041.0,418.0,92.0,37.08,94.83,239.666667,263.615385,0.314815,1.555556,22.722222,6.166667,0.438,6.0,28.71,31.5,5.46,215.388889,7.500000
1,Sri Lanka,900.0,443.0,73.0,27.23,82.89,211.645833,263.615385,0.187500,1.250000,20.250000,2.645833,0.351,8.0,30.17,33.8,5.34,216.875000,7.187500
2,Bangladesh,423.0,349.0,77.0,28.64,79.20,203.531915,263.615385,0.191489,1.212766,18.808511,3.063830,0.390,15.0,27.63,33.7,4.91,201.702128,7.297872
3,New Zealand,811.0,402.0,59.0,31.39,88.09,223.794872,263.615385,0.461538,0.948718,20.846154,4.641026,0.490,10.0,31.59,35.2,5.38,229.256410,7.256410
4,South Africa,659.0,439.0,53.0,36.23,96.48,247.235294,263.615385,0.470588,1.205882,23.058824,5.941176,0.478,11.0,33.27,34.6,5.75,237.794118,7.147059
5,Pakistan,961.0,399.0,56.0,35.36,90.21,242.323529,263.615385,0.500000,1.352941,21.529412,4.441176,0.448,4.0,31.20,34.1,5.48,246.000000,7.882353
6,Australia,986.0,434.0,57.0,31.69,94.32,237.676471,263.615385,0.294118,1.352941,23.029412,5.705882,0.448,3.0,31.03,34.7,5.35,236.441176,7.617647
7,England,785.0,498.0,49.0,34.51,100.21,239.545455,263.615385,0.454545,1.151515,22.060606,6.575758,0.481,5.0,31.98,34.7,5.52,229.696970,7.181818
8,Netherlands,114.0,374.0,46.0,23.67,76.51,200.500000,263.615385,0.117647,1.205882,17.235294,3.147059,0.368,6.0,40.21,44.5,5.41,244.852941,6.088235
9,Afghanistan,152.0,338.0,39.0,28.75,80.12,211.230769,263.615385,0.384615,1.115385,19.000000,4.846154,0.286,2.0,31.34,37.9,4.95,217.000000,6.923077


In [576]:
teamIndices = ['India', 'Sri Lanka', 'Bangladesh', 'New Zealand', 'South Africa', 'Pakistan', 'Australia', 'England', 'Netherlands', 'Afghanistan']
stats=['MatPlayed', 'High_inns_score', 'NO', 'BatAvg', 'Bat_SR',
       'Runs/Inns', 'BF/Inns', 'Hundreds/Inns', 'Fifties/Inns',
       'Boundries/Inns', 'Sixes/Inns', 'FieldingDiss/Inns', 'Stumping',
       'BallAvg', 'Ball_SR', 'Econ', 'RunsConceded/Inns', 'WickTaken/Inns']
i = 1
for stat in stats:
    arr = []
    for index,row in match_pred.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_2"])][i])
    i+=1
    match_pred[stat + '2'] = arr

i = 1
for stat in stats:
    arr = []
    for index,row in match_pred.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_1"])][i])
    i+=1
    match_pred[stat + '1'] = arr

In [577]:
pmatch=match_pred.copy()
T1 = []
T2 = []
for index,row in pmatch.iterrows():
    T1.append(teamIndices.index(row["Team_1"]))
    T2.append(teamIndices.index(row["Team_2"]))
pmatch["Team_1"] = T1
pmatch["Team_2"] = T2

In [578]:
X=pmatch.drop(['Winner'],axis=1)
X['Year']=X['Year'].astype('int')
y=pmatch['Winner'].astype('int')

In [579]:
ss=MinMaxScaler()
X=pd.DataFrame(ss.fit_transform(X),columns=X.columns)


In [580]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.15)

In [581]:
#xgBoost predictor, test for auc score and test error (Best AUC Score = 0.717, Best test error = 21.8%)

xg_reg1 = xgb.XGBRegressor(objective ='binary:logistic', colsample_bytree = 0.30, learning_rate = 0.01, max_depth =100,
                          alpha = 0, n_estimators = 500, min_child_weight = 2.15, gamma = 0.20)
xg_reg1.fit(x_train,y_train)
preds = xg_reg1.predict(x_test)
error = roc_auc_score(y_test,preds)


for i in range(0,len(preds)):
    if (preds[i]>0.5):
        preds[i]=1
    else:
        preds[i]=0

print(error)

print('Classification Report\n',classification_report(preds,y_test))


0.6450014573010784
Classification Report
               precision    recall  f1-score   support

         0.0       0.78      0.69      0.73        83
         1.0       0.45      0.57      0.50        37

    accuracy                           0.65       120
   macro avg       0.61      0.63      0.62       120
weighted avg       0.68      0.65      0.66       120



In [582]:
matches.head()

,Team_1,Team_2,Winner,Margin,Year,WinRateVsTeam,Team_1_WR,Team_2_WR,WinByWickets
0,Bangladesh,Sri Lanka,Sri Lanka,7,2010,0.0,0.00,0.75,1
1,India,Sri Lanka,Sri Lanka,5,2010,0.5,0.75,0.75,1
2,Bangladesh,India,India,6,2010,0.0,0.00,0.75,1
3,Bangladesh,Sri Lanka,Sri Lanka,9,2010,0.0,0.00,0.75,1
4,India,Sri Lanka,India,8,2010,0.5,0.75,0.75,1


In [583]:
margin0=matches.loc[matches['WinByWickets']==0]
margin1=matches.loc[matches['WinByWickets']==1]

In [584]:
i = 1
for stat in stats:
    arr = []
    for index,row in margin0.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_2"])][i])
    i+=1
    margin0[stat + '2'] = arr

i = 1
for stat in stats:
    arr = []
    for index,row in margin0.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_1"])][i])
    i+=1
    margin0[stat + '1'] = arr

i = 1
for stat in stats:
    arr = []
    for index,row in margin1.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_2"])][i])
    i+=1
    margin1[stat + '2'] = arr

i = 1
for stat in stats:
    arr = []
    for index,row in margin1.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_1"])][i])
    i+=1
    margin1[stat + '1'] = arr

In [585]:
T1_0 = []
T2_0 = []
Winner_0 = []
for index,row in margin0.iterrows():
    T1_0.append(teamIndices.index(row["Team_1"]))
    T2_0.append(teamIndices.index(row["Team_2"]))
    Winner_0.append(teamIndices.index(row["Winner"]))
margin0["Team_1"] = T1_0
margin0["Team_2"] = T2_0
margin0["Winner"] = Winner_0

marginMatches0 = margin0.astype("float64")
T1_1 = []
T2_1 = []
Winner_1 = []
for index,row in margin1.iterrows():
    T1_1.append(teamIndices.index(row["Team_1"]))
    T2_1.append(teamIndices.index(row["Team_2"]))
    Winner_1.append(teamIndices.index(row["Winner"]))
margin1["Team_1"] = T1_1
margin1["Team_2"] = T2_1
margin1["Winner"] = Winner_1

marginMatches1 = margin1.astype("float64")

In [586]:
X_0=margin0.drop(['Margin','WinByWickets'],axis=1)
X_0['Year']=X_0['Year'].astype('int')
y_0=margin0['Margin']
from sklearn.model_selection import train_test_split
x_0_train,x_0_test,y_0_train,y_0_test=train_test_split(X_0,y_0,test_size=0.15)

In [587]:
X_1=margin1.drop(['Margin','WinByWickets'],axis=1)
y_1=margin1['Margin']
X_1['Year']=X_1['Year'].astype('int')
from sklearn.model_selection import train_test_split
x_1_train,x_1_test,y_1_train,y_1_test=train_test_split(X_1,y_1,test_size=0.15)

In [588]:
xg_regMarg0 = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.018, max_depth = 15,
                          alpha = 0, n_estimators = 110, subsample = 0.85, min_child_weight = 4, gamma = 2)

xg_regMarg0.fit(x_0_train,y_0_train)
predsMarg0 = xg_regMarg0.predict(x_0_test)
error = mean_squared_error(y_0_test,predsMarg0)

print("error = " + str(error))

[10:32:35] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
error = 3818.770423061558


In [589]:

xg_regMarg1 = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.34, learning_rate = 0.017, max_depth = 6,
                          alpha = 0, n_estimators = 229, subsample = 0.85, min_child_weight = 5, gamma = 0.24)


xg_regMarg1.fit(x_1_train,y_1_train)
predsMarg1 = xg_regMarg1.predict(x_1_test)
error = mean_squared_error(y_1_test,predsMarg1)

print("error = " + str(error))

[10:32:35] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
error = 4.468707647355037


In [590]:
final=final.drop(['WinRateVsTeam','Team_1_WR','Team_2_WR'],axis=1)
final

,Team_1,Team_2,Winner,Margin,Year,WinByWickets
0,Bangladesh,Sri Lanka,Sri Lanka,7,2010,1
1,India,Sri Lanka,Sri Lanka,5,2010,1
2,Bangladesh,India,India,6,2010,1
3,Bangladesh,Sri Lanka,Sri Lanka,9,2010,1
4,India,Sri Lanka,India,8,2010,1
...,...,...,...,...,...,...
789,Pakistan,New Zealand,Pakistan,5,2023,1
790,Pakistan,New Zealand,Pakistan,7,2023,1
791,Pakistan,New Zealand,Pakistan,26,2023,0
792,Pakistan,New Zealand,Pakistan,102,2023,0


In [591]:
preds = xg_reg1.predict(X)
for i in range(0,len(preds)):
    if (preds[i]>0.5):
        preds[i]=1
    else:
        preds[i]=0

final['Predicted Winner']=preds

In [592]:
final["Winner_Pred"]=final['Predicted Winner']
for i in range(0,len(final)):
  if final['Predicted Winner'][i]== 0.0:
    final["Winner_Pred"][i]=final["Team_1"][i]
  else:
    final["Winner_Pred"][i]=final["Team_2"][i]
final=final.drop(['Predicted Winner'],axis=1)

In [593]:
fpred=matches.copy()

In [594]:
i = 1
for stat in stats:
    arr = []
    for index,row in fpred.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_2"])][i])
    i+=1
    fpred[stat + '2'] = arr

i = 1
for stat in stats:
    arr = []
    for index,row in fpred.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_1"])][i])
    i+=1
    fpred[stat + '1'] = arr

In [595]:
T1_0 = []
T2_0 = []
Winner_0 = []
for index,row in fpred.iterrows():
    T1_0.append(teamIndices.index(row["Team_1"]))
    T2_0.append(teamIndices.index(row["Team_2"]))
    Winner_0.append(teamIndices.index(row["Winner"]))
fpred["Team_1"] = T1_0
fpred["Team_2"] = T2_0
fpred["Winner"] = Winner_0

fpred = fpred.astype("float64")

In [596]:
fpred=fpred.drop(['Margin','WinByWickets'],axis=1)

In [597]:
preds0 = xg_regMarg0.predict(fpred)
preds1 = xg_regMarg1.predict(fpred)

In [598]:
final['Run_Margin_pred']=preds0
final['Wick_Margin_pred']=preds1

In [599]:
for i in range(0,len(final)):
  final['Run_Margin_pred'][i]=str(round(final['Run_Margin_pred'][i])) +' Runs'
  final['Wick_Margin_pred'][i]=str(round(final['Wick_Margin_pred'][i]))+' Wickets'

In [600]:
for i in range(0,len(final)):
  if final['WinByWickets'][i]==1:
    j=str(final['Margin'][i]) +' Wickets'
    final['Margin'][i]=j
  else:
    j=str(final['Margin'][i]) +' Runs'
    final['Margin'][i]=j


In [601]:
final=final.drop(['WinByWickets'], axis=1)

In [602]:
df = final.reindex(columns=['Team_1', 'Team_2', 'Winner','Winner_Pred','Margin','Run_Margin_pred', 'Wick_Margin_pred','Year'])

In [603]:
df

,Team_1,Team_2,Winner,Winner_Pred,Margin,Run_Margin_pred,Wick_Margin_pred,Year
0,Bangladesh,Sri Lanka,Sri Lanka,Sri Lanka,7 Wickets,67 Runs,6 Wickets,2010
1,India,Sri Lanka,Sri Lanka,India,5 Wickets,83 Runs,5 Wickets,2010
2,Bangladesh,India,India,India,6 Wickets,69 Runs,6 Wickets,2010
3,Bangladesh,Sri Lanka,Sri Lanka,Sri Lanka,9 Wickets,67 Runs,6 Wickets,2010
4,India,Sri Lanka,India,India,8 Wickets,90 Runs,5 Wickets,2010
...,...,...,...,...,...,...,...,...
789,Pakistan,New Zealand,Pakistan,Pakistan,5 Wickets,54 Runs,5 Wickets,2023
790,Pakistan,New Zealand,Pakistan,Pakistan,7 Wickets,54 Runs,5 Wickets,2023
791,Pakistan,New Zealand,Pakistan,Pakistan,26 Runs,54 Runs,5 Wickets,2023
792,Pakistan,New Zealand,Pakistan,Pakistan,102 Runs,54 Runs,5 Wickets,2023


In [604]:
df.to_csv("BasicModelPredictions.csv")

In [605]:
wc=pd.read_csv('/content/Fixtures.csv')

In [606]:
wc=wc.drop(['Round_number','Group','Result'],axis=1)

In [607]:
wc=wc.drop([45,46,47],axis=0)
wc

,Team_1,Team_2,Date,Location
0,England,New Zealand,5/10/2023,"Narendra Modi Stadium, Ahmedabad"
1,Pakistan,Netherlands,6/10/2023,"Rajiv Gandhi International Stadium, Hyderabad"
2,Bangladesh,Afghanistan,7/10/2023,"Himachal Pradesh Cricket Association Stadium, ..."
3,South Africa,Sri Lanka,7/10/2023,"Arun Jaitley Stadium, Delhi"
4,India,Australia,8/10/2023,"MA Chidambaram Stadium, Chennai"
5,New Zealand,Netherlands,9/10/2023,"Rajiv Gandhi International Stadium, Hyderabad"
6,England,Bangladesh,10/10/2023,"Himachal Pradesh Cricket Association Stadium, ..."
7,India,Afghanistan,11/10/2023,"Arun Jaitley Stadium, Delhi"
8,Pakistan,Sri Lanka,12/10/2023,"Rajiv Gandhi International Stadium, Hyderabad"
9,Australia,South Africa,13/10/2023,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...


In [608]:
WCup=wc.copy()
WCup=WCup.drop(['Date','Location'],axis=1)
WCup["Year"]=2023

In [609]:
team=[]
for i in WCup['Team_1']:
  sp=i.split(' ')
  if sp[0]=='Pakistan':
    team.append(sp[0])
  else:
    team.append(i)
WCup['Team_1']=team

In [610]:
winrates = []
for index, row in WCup.iterrows():
    Team_1 = row["Team_1"]
    Team_2 = row["Team_2"]
    count1 = 0
    count2 = 0
    for i in range(0,len(row)):
        if((matches["Team_1"][i] == Team_1 and matches["Team_2"][i] == Team_2)
          or (matches["Team_1"][i] == Team_2 and matches["Team_2"][i] == Team_1)):
            if(matches["Winner"][i] == Team_1):
                count1 += int(matches["Year"][i]) - 2009
            else:
                count2 += int(matches["Year"][i]) - 2009
    if(count1 + count2 > 0):
        winrates.append(round(count1/(count1+count2),2))
    else:
        winrates.append(0.50)
WCup["WinRateVsTeam"] = winrates

In [611]:
winratesT1 = []
winratesT2=[]
for index, row in WCup.iterrows():
    Team_1 = row["Team_1"]
    Team_2 = row["Team_2"]
    count1_t1 = 0
    count2_t1= 0
    count1_t2 = 0
    count2_t2= 0
    for i in range(0,len(row)):
      if(matches["Team_1"][i] == Team_1  or matches["Team_2"][i] == Team_1):
            if(matches["Winner"][i] == Team_1):
                count1_t1 += int(matches["Year"][i]) - 2009
            else:
                count2_t1 += int(matches["Year"][i]) - 2009
      if(matches["Team_1"][i] == Team_2  or matches["Team_2"][i] == Team_2):
            if(matches["Winner"][i] == Team_2):
                count1_t2 += int(matches["Year"][i]) - 2009
            else:
                count2_t2 += int(matches["Year"][i]) - 2009

    if(count1_t1 + count2_t1 > 0):
        winratesT1.append(round(count1_t1/(count1_t1+count2_t1),2))
    else:
        winratesT1.append(0.50)


    if(count1_t2 + count2_t2 > 0):
        winratesT2.append(round(count1_t2/(count1_t2+count2_t2),2))
    else:
        winratesT2.append(0.50)



WCup["Team_1_WR"] = winratesT1
WCup["Team_2_WR"] = winratesT2

In [612]:
i = 1
for stat in stats:
    arr = []
    for index,row in WCup.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_2"])][i])
    i+=1
    WCup[stat + '2'] = arr
i = 1
for stat in stats:
    arr = []
    for index,row in WCup.iterrows():
        arr.append(teamData.loc[teamIndices.index(row["Team_1"])][i])
    i+=1
    WCup[stat + '1'] = arr

In [613]:
pmatch=WCup.copy()
T1 = []
T2 = []
for index,row in pmatch.iterrows():
    T1.append(teamIndices.index(row["Team_1"]))
    T2.append(teamIndices.index(row["Team_2"]))
pmatch["Team_1"] = T1
pmatch["Team_2"] = T2

In [614]:
xg_reg1.fit(X,y)
preds = xg_reg1.predict(pmatch)

for i in range(0,len(preds)):
    if (preds[i]>0.5):
        preds[i]=1
    else:
        preds[i]=0

In [615]:
preds

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [616]:
pmatch.insert(2,'Winner',preds)

In [617]:
xg_regMarg0 = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.018, max_depth = 15,
                          alpha = 0, n_estimators = 110, subsample = 0.85, min_child_weight = 4, gamma = 2)

xg_regMarg0.fit(X_0,y_0)

predsMarg0 = xg_regMarg0.predict(pmatch)

xg_regMarg1 = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.34, learning_rate = 0.017, max_depth = 6,
                          alpha = 0, n_estimators = 229, subsample = 0.85, min_child_weight = 5, gamma = 0.24)


xg_regMarg1.fit(X_1,y_1)
predsMarg1 = xg_regMarg1.predict(pmatch)

[10:32:40] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[10:32:40] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [618]:
wc['Winner']=preds
wc['Run Margin']=predsMarg0
wc['Wicket Margin']=predsMarg1

In [619]:
for i in range(0,len(wc)):
  if wc['Winner'][i]== 0.0:
    wc["Winner"][i]=wc["Team_1"][i]
  else:
    wc["Winner"][i]=wc["Team_2"][i]

In [620]:
for i in range(0,len(wc)):
  wc['Run Margin'][i]=str(round(wc['Run Margin'][i])) +' Runs'
  wc['Wicket Margin'][i]=str(round(wc['Wicket Margin'][i]))+' Wickets'

In [621]:
wc

,Team_1,Team_2,Date,Location,Winner,Run Margin,Wicket Margin
0,England,New Zealand,5/10/2023,"Narendra Modi Stadium, Ahmedabad",New Zealand,58 Runs,5 Wickets
1,Pakistan,Netherlands,6/10/2023,"Rajiv Gandhi International Stadium, Hyderabad",Pakistan,81 Runs,6 Wickets
2,Bangladesh,Afghanistan,7/10/2023,"Himachal Pradesh Cricket Association Stadium, ...",Bangladesh,78 Runs,5 Wickets
3,South Africa,Sri Lanka,7/10/2023,"Arun Jaitley Stadium, Delhi",South Africa,92 Runs,6 Wickets
4,India,Australia,8/10/2023,"MA Chidambaram Stadium, Chennai",India,36 Runs,7 Wickets
5,New Zealand,Netherlands,9/10/2023,"Rajiv Gandhi International Stadium, Hyderabad",New Zealand,106 Runs,7 Wickets
6,England,Bangladesh,10/10/2023,"Himachal Pradesh Cricket Association Stadium, ...",England,82 Runs,7 Wickets
7,India,Afghanistan,11/10/2023,"Arun Jaitley Stadium, Delhi",India,106 Runs,6 Wickets
8,Pakistan,Sri Lanka,12/10/2023,"Rajiv Gandhi International Stadium, Hyderabad",Pakistan,62 Runs,5 Wickets
9,Australia,South Africa,13/10/2023,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,South Africa,38 Runs,5 Wickets


In [622]:
wc.to_csv('WorldCupPredictions.csv')